In [ ]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.10" 2021-01-19
OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)


In [ ]:
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-2.4.7-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark
!pip install -q elephas
!pip install --ignore-installed PyYAML

     |████████████████████████████████| 337kB 19.8MB/s 
     |████████████████████████████████| 212.3MB 74kB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 204kB 57.5MB/s 
     |████████████████████████████████| 645kB 18.2MB/s 


In [ ]:
import os
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('Elephas_App').setMaster('local[8]')
sc = SparkContext(conf=conf)

In [ ]:
from __future__ import print_function
print(sc)

<SparkContext master=local[8] appName=Elephas_App>


In [ ]:
from google.colab import files
files.upload()

Saving CS551G_DMV_Assessment_2_Dataset.xlsx to CS551G_DMV_Assessment_2_Dataset.xlsx


{'CS551G_DMV_Assessment_2_Dataset.xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00\x87V\xe12\x86\x01\x00\x00\x99\x06\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [ ]:
data_path = "./"

In [ ]:
import pandas as pd

# load excel with pandas
p_df = pd.read_excel('/content/CS551G_DMV_Assessment_2_Dataset.xlsx')
p_df = p_df.rename(columns={'Flow_Rate_Sensor_1.1':'Flow_Rate_Sensor_2'})
p_df = p_df.rename(columns={'Flow_Rate_Sensor_1.2':'Flow_Rate_Sensor_3'})
p_df = p_df.sample(frac=1.0)    # randonly shuffle the data
rows, _ = p_df.shape
split_index = int(rows * 0.7)

# split the data
train_df = p_df.iloc[0: split_index, :]
test_df = p_df.iloc[split_index:rows, :]

# drop 'Status' column of test_df
test_df = test_df.drop(columns=['Status'])

# move 'Status' column at the tail of train_df
_, col = train_df.shape
c = train_df.pop('Status')
train_df.insert(col-1, 'Status', c)

# add id column
test_df.insert(0, 'id', range(len(test_df)), allow_duplicates=False)
train_df.insert(0, 'id', range(len(train_df)), allow_duplicates=False)

# save the files to csv
train_df.to_csv('train.csv', index=False, header=True)
test_df.to_csv('test.csv', index=False, header=True)

In [ ]:
from pyspark.sql import SQLContext
from pyspark.ml.linalg import Vectors
import numpy as np
import random

sql_context = SQLContext(sc)

def shuffle_csv(csv_file):
    lines = open(csv_file).readlines()
    random.shuffle(lines)
    open(csv_file, 'w').writelines(lines)

def load_data_frame(csv_file, shuffle=True, train=True):
    if shuffle:
        shuffle_csv(csv_file)
    data = sc.textFile(data_path + csv_file) # This is an RDD, which will later be transformed to a data frame
    data = data.filter(lambda x:x.split(',')[0] != 'id').map(lambda line: line.split(','))
    if train:
        data = data.map(
            lambda line: (Vectors.dense(np.asarray(line[1:-1]).astype(np.float32)),
                          str(line[-1])) )
    else:
        # Test data gets dummy labels. We need the same structure as in Train data
        data = data.map( lambda line: (Vectors.dense(np.asarray(line[1:]).astype(np.float32)),"Normal") ) 
    return sql_context.createDataFrame(data, ['features', 'Status'])

In [ ]:
train_df = load_data_frame('train.csv')
test_df = load_data_frame('test.csv', shuffle=False, train=False)

print("Train data frame:")
train_df.show(10)
train_df.printSchema()

print("Test data frame (note the dummy category):")
test_df.show(10)

train_df.head(1)

Train data frame:
+--------------------+--------+
|            features|  Status|
+--------------------+--------+
|[4.25085020065307...|Abnormal|
|[3.49909996986389...|  Normal|
|[6.50209999084472...|  Normal|
|[2.09750008583068...|  Normal|
|[3.27399611473083...|Abnormal|
|[4.22970008850097...|  Normal|
|[5.40439987182617...|  Normal|
|[4.73969984054565...|  Normal|
|[1.65382802486419...|Abnormal|
|[7.35080003738403...|Abnormal|
+--------------------+--------+
only showing top 10 rows

root
 |-- features: vector (nullable = true)
 |-- Status: string (nullable = true)

Test data frame (note the dummy category):
+--------------------+------+
|            features|Status|
+--------------------+------+
|[0.25030800700187...|Normal|
|[8.70940017700195...|Normal|
|[8.42010021209716...|Normal|
|[3.27119994163513...|Normal|
|[8.31480026245117...|Normal|
|[6.37389993667602...|Normal|
|[3.01440596580505...|Normal|
|[6.73760986328125...|Normal|
|[1.62870001792907...|Normal|
|[1.55359995365142...

[Row(features=DenseVector([4.2509, 5.1986, 7.2493, 4.4236, 26.6977, 0.4386, 6.6477, 1.1555, 9.5198, 23.1074, 7.2411, 9.5198]), Status='Abnormal')]

In [ ]:
from pyspark.ml.feature import StringIndexer

string_indexer = StringIndexer(inputCol="Status", outputCol="index_Status")
fitted_indexer = string_indexer.fit(train_df)
indexed_df = fitted_indexer.transform(train_df)

In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
fitted_scaler = scaler.fit(indexed_df)
scaled_df = fitted_scaler.transform(indexed_df)

In [ ]:
print("The result of indexing and scaling. Each transformation adds new columns to the data frame.")
scaled_df.show(10)

The result of indexing and scaling. Each transformation adds new columns to the data frame.
+--------------------+--------+------------+--------------------+
|            features|  Status|index_Status|     scaled_features|
+--------------------+--------+------------+--------------------+
|[3.054594,7.59400...|Abnormal|         1.0|[-0.6776399139282...|
|[3.109674,8.44376...|Abnormal|         1.0|[-0.6578446376321...|
|[4.53,6.3968,12.3...|Abnormal|         1.0|[-0.1473917400170...|
|[7.7151,7.6117,6....|  Normal|         0.0|[0.99730569155407...|
|[2.99013,8.439072...|Abnormal|         1.0|[-0.7008077187786...|
|[12.1298,5.387,10...|  Normal|         0.0|[2.58391074356410...|
|[5.8694,4.2938,9....|Abnormal|         1.0|[0.33397705212333...|
|[5.5225,6.0992,9....|Abnormal|         1.0|[0.20930418815924...|
|[3.8882,9.4107,10...|  Normal|         0.0|[-0.3780491170483...|
|[7.6144,7.8461,9....|  Normal|         0.0|[0.96111498125951...|
+--------------------+--------+------------+------

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation
from tensorflow.python.keras.utils import np_utils, generic_utils

nb_classes = train_df.select("Status").distinct().count()   # 2 classes
input_dim = len(train_df.select("features").first()[0])

model = Sequential()
model.add(Dense(512, input_shape=(input_dim,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
from elephas.ml_model import ElephasEstimator
from tensorflow.keras import optimizers

adam = optimizers.Adam(lr=0.003)
opt_conf = optimizers.serialize(adam)

# Initialize SparkML Estimitor and set all relevant properties
estimator = ElephasEstimator()
estimator.setFeaturesCol("scaled_features")
estimator.setLabelCol("index_Status")
estimator.set_keras_model_config(model.to_yaml())
estimator.set_categorical_labels(True)
estimator.set_nb_classes(nb_classes)
estimator.set_num_workers(2)
estimator.set_epochs(40)
estimator.set_batch_size(128)
estimator.set_verbosity(1)
estimator.set_validation_split(0.15)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("categorical_crossentropy")
estimator.set_metrics(['acc'])

ElephasEstimator_2c4f8a6e015f

In [ ]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[string_indexer, scaler, estimator])

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics

fitted_pipeline = pipeline.fit(train_df)

prediction = fitted_pipeline.transform(train_df)
# prediction = fitted_pipeline.transform(test_df)

pnl = prediction.select("index_Status", "prediction")

pnl.show(100)

prediction1_and_label = pnl.rdd.map(lambda row: (row.index_Status, row.prediction))
metrics = MulticlassMetrics(prediction1_and_label)
# print(metrics.precision())

>>> Fit model
>>> Synchronous training complete.
+------------+--------------------+
|index_Status|          prediction|
+------------+--------------------+
|         1.0|[0.07423058152198...|
|         1.0|[0.21709659695625...|
|         0.0|[0.94231098890304...|
|         1.0|[1.17160408990457...|
|         1.0|[0.00207798345945...|
|         1.0|[0.10999401658773...|
|         0.0|[0.87173652648925...|
|         1.0|[0.44925847649574...|
|         0.0|[0.94360321760177...|
|         1.0|[0.65069752931594...|
|         0.0|[0.79311126470565...|
|         1.0|[0.01574497111141...|
|         1.0|[0.02992281317710...|
|         1.0|[0.85060328245162...|
|         1.0|[0.12688443064689...|
|         0.0|[0.80256026983261...|
|         0.0|[0.88509386777877...|
|         1.0|[0.05394018813967...|
|         1.0|[0.15489630401134...|
|         0.0|[0.99093854427337...|
|         1.0|[0.61161828041076...|
|         0.0|[0.87682777643203...|
|         0.0|[0.81840211153030...|
|         1.0|[

In [ ]:
print(metrics.accuracy())

Py4JJavaError: ignored

In [ ]:
pnl.head(1)

[Row(index_Status=1.0, prediction=[0.07423058152198792, 0.9257694482803345])]